In [ ]:
import os
if os.getcwd().split("/")[-1] == "notebooks":
    os.chdir(os.pardir)

import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import open3d as o3d
import seaborn as sns

from src.utils.dataloader import load_ear_data
from src.utils.viz import set_axes_equal

In [ ]:
sns.set(style='ticks', palette='colorblind')

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
PROJECT_NAME = 'IMBioC2022_paper'

In [ ]:
def clean_df(df):
    df = df[(df['x [mm]'] != df['x [mm]'].min())
            & (df['x [mm]'] != df['x [mm]'].max())
            & (df['y [mm]'] != df['y [mm]'].min())
            & (df['y [mm]'] != df['y [mm]'].max())
            & (df['z [mm]'] != df['z [mm]'].min())
            & (df['z [mm]'] != df['z [mm]'].max())]
    df.reset_index(drop=True, inplace=True)
    return df


def export_pcd(df, area=False):
    if area:
        pcd = np.c_[df['x [mm]'].to_numpy(),
                    df['y [mm]'].to_numpy(),
                    df['z [mm]'].to_numpy(),
                    df['area [mm^2]'].to_numpy()]
    else:
        pcd = np.c_[df['x [mm]'].to_numpy(),
                    df['y [mm]'].to_numpy(),
                    df['z [mm]'].to_numpy()]
    return pcd


def export_fields(df):
    Ex = df['ExRe [V/m]'].to_numpy() + 1j * df['ExIm [V/m]'].to_numpy()
    Ey = df['EyRe [V/m]'].to_numpy() + 1j * df['EyIm [V/m]'].to_numpy()
    Ez = df['EzRe [V/m]'].to_numpy() + 1j * df['EzIm [V/m]'].to_numpy()
    Hx = df['HxRe [A/m]'].to_numpy() + 1j * df['HxIm [A/m]'].to_numpy()
    Hy = df['HyRe [A/m]'].to_numpy() + 1j * df['HyIm [A/m]'].to_numpy()
    Hz = df['HzRe [A/m]'].to_numpy() + 1j * df['HzIm [A/m]'].to_numpy()
    return ((Ex, Ey, Ez), (Hx, Hy, Hz))


def poynting_vector(E, H):
    return (E[1] * H[2].conjugate() - E[2] * H[1].conjugate(),
            E[2] * H[0].conjugate() - E[0] * H[2].conjugate(),
            E[0] * H[1].conjugate() - E[1] * H[0].conjugate())


def plot_2d(xy_dict, figsize=plt.rcParams['figure.figsize'], c=None, alpha=1):
    fig = plt.figure(figsize=figsize)
    ax = plt.axes()
    keys = list(xy_dict.keys())
    values = list(xy_dict.values())
    if (len(values) == 3) and not(c):
        cs = ax.scatter(values[0], values[1], c=values[2], cmap='viridis')
        cbar = fig.colorbar(cs)
        cbar.ax.set_ylabel(keys[2])
    else:
        if not(c):
            c = 'k'
        cs = ax.scatter(values[0], values[1], c=c, alpha=alpha)
    ax.set(xlabel=keys[0], ylabel=keys[1])
    
    ax.axis('equal')
    fig.tight_layout()
    return fig, ax


def plot_3d(xyz_dict, figsize=(7, 7), elev=20, azim=45, c=None, alpha=1):
    fig = plt.figure(figsize=figsize)
    ax = plt.axes(projection ='3d')
    keys = list(xyz_dict.keys())
    values = list(xyz_dict.values())
    if (len(values) == 4) and not(c):
        cs = ax.scatter(values[0], values[1], values[2],
                        c=values[3], cmap='viridis')
        cbar = fig.colorbar(cs, shrink=0.5)
        cbar.ax.set_ylabel(keys[3])
    else:
        if not(c):
            c = 'k'
        cs = ax.plot(values[0], values[1], values[2], '.', c=c, alpha=alpha)
    ax.set(xlabel=keys[0], ylabel=keys[1], zlabel=keys[2])
    ax = set_axes_equal(ax)
    ax.view_init(elev, azim)
    fig.tight_layout()
    return fig, ax


def estimate_normals(xyz, knn, down_sampling_ratio=1, orient_normals=False):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.paint_uniform_color(np.array([1, 0, 0]))
    pcd = pcd.random_down_sample(down_sampling_ratio)
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn))
    if orient_normals is True:
        pcd.orient_normals_consistent_tangent_plane(knn)
    elif orient_normals:
        pcd.orient_normals_consistent_tangent_plane(int(orient_normals))
    return np.asarray(pcd.points), np.asarray(pcd.normals)

In [ ]:
df = load_ear_data('te', 60)
df = clean_df(df)
df

In [ ]:
xyz = export_pcd(df)
xyz.shape

In [ ]:
E, H = export_fields(df)
len(E), len(H)

In [ ]:
Sx, Sy, Sz = poynting_vector(E, H)
S_dist_cpx = np.sqrt(Sx ** 2 + Sy ** 2 + Sz ** 2)
S_dist_abs = np.abs(S_dist_cpx)

In [ ]:
fig, ax = plot_3d({'x [mm]': xyz[:, 0],
                   'y [mm]': xyz[:, 1],
                   'z [mm]': xyz[:, 2],
                   'S [W/m2]': S_dist_abs[:]})

In [ ]:
# downsample dataset
skip = 1
xyz_ds = xyz[::skip, :]
S_dist_abs_ds = S_dist_abs[::skip]

In [ ]:
# define coordinate frame
cframe = o3d.geometry.TriangleMesh.create_coordinate_frame(size=10,
                                                           origin=[0] * 3)
# define colors for open3d
colors = cm.viridis((S_dist_abs_ds - S_dist_abs_ds.min())
                    / (S_dist_abs_ds.max() - S_dist_abs_ds.min()))[:, :3]
color_vec = o3d.utility.Vector3dVector(colors)

In [ ]:
# create opend3d pcd
pcd_ds = o3d.geometry.PointCloud()
pcd_ds.points = o3d.utility.Vector3dVector(xyz_ds)
pcd_ds.colors = color_vec

o3d.visualization.draw_geometries([pcd_ds, cframe])

In [ ]:
# translate pcd to have a (0, 0, 0) center
center = pcd_ds.get_center()
xyz_ds_t = np.c_[xyz_ds[:, 0] - center[0],
                 xyz_ds[:, 1] - center[1],
                 xyz_ds[:, 2] - center[2]]
pcd_ds_t = o3d.geometry.PointCloud()
pcd_ds_t.points = o3d.utility.Vector3dVector(xyz_ds_t)
pcd_ds_t.colors = color_vec

o3d.visualization.draw_geometries([pcd_ds_t, cframe])

In [ ]:
# select x-visible indices
diameter = np.linalg.norm(pcd_ds_t.get_max_bound() - pcd_ds_t.get_min_bound())
radius = 10 ** 6
camera = [diameter, 0, 0]

_, pt_map = pcd_ds_t.hidden_point_removal(camera, radius)
pcd_ds_t_visible = pcd_ds_t.select_by_index(pt_map)

o3d.visualization.draw_geometries([pcd_ds_t_visible, cframe])

In [ ]:
# estimate normals on x-visible indices
pcd_ds_t_visible.estimate_normals()
pcd_ds_t_visible.orient_normals_consistent_tangent_plane(30)

o3d.visualization.draw_geometries([pcd_ds_t_visible], point_show_normal=True)